# Semantic LLM caching

_**NOTE:** this uses the experimental "Vector Similarity Search" capability.
At the moment, this is obtained by building and running an early alpha from a specific branch of the codebase._

In [1]:
from langchain.cache import CassandraSemanticCache

/home/stefano/.virtualenvs/langchain-cassio-3.10/lib/python3.10/site-packages/cassandra/io/asyncorereactor.py:34: DeprecationWarning: The asyncore module is deprecated and will be removed in Python 3.12. The recommended replacement is asyncio
  import asyncore


In [2]:
from cqlsession import getLocalCassandraSession
localSession = getLocalCassandraSession()

In [3]:
# Open AI
from langchain.llms import OpenAI
from langchain.embeddings import OpenAIEmbeddings
myEmbedding = OpenAIEmbeddings()
llm = OpenAI(model_name="text-davinci-002", n=2, best_of=2)

In [4]:
from langchain.llms.base import get_prompts
this_llm_string = get_prompts(llm.dict(), [])[1]

In [5]:
import langchain

In [6]:
# For the time being we have to explicitly TURN ON THIS EXPERIMENTAL FLAG on the cassio side
import cassio
cassio.globals.enableExperimentalVectorSearch()

In [7]:
cassSemanticCache = CassandraSemanticCache(
    session=localSession,
    keyspace='demo',
    embedding=myEmbedding
)

In [8]:
cassSemanticCache.clear(llm_string=this_llm_string)

In [9]:
langchain.llm_cache = cassSemanticCache

In [10]:
%%time
# The first time, it is not yet in cache, so it should take longer
llm("How many eyes do spiders have?")

CPU times: user 14.1 ms, sys: 3.11 ms, total: 17.2 ms
Wall time: 507 ms


'\n\nMost spiders have eight eyes.'

In [11]:
%%time
# Second time, very same question
llm("How many eyes do spiders have?")

CPU times: user 4.43 ms, sys: 197 µs, total: 4.62 ms
Wall time: 55.1 ms


'\n\nMost spiders have eight eyes.'

In [12]:
%%time
# Rephrased, but same as above!
llm("How many eyes does a spider generally have?")

CPU times: user 11.3 ms, sys: 7.69 ms, total: 19 ms
Wall time: 388 ms


'\n\nMost spiders have eight eyes.'

In [13]:
%%time
# New question
llm("Is absence of proof the same as proof of absence?")

CPU times: user 23.4 ms, sys: 1.88 ms, total: 25.3 ms
Wall time: 1.32 s


'\n\nNo, the two are not the same. Proof of absence would require evidence that something does not exist, while the absence of proof only means that there is no evidence to support something.'

In [14]:
%%time
# Trying to catch the cache off-guard
llm("How many eyes are on the head of a typical spider?")

CPU times: user 17.3 ms, sys: 0 ns, total: 17.3 ms
Wall time: 334 ms


'\n\nMost spiders have eight eyes.'

In [15]:
%%time
# Switching to the other question again
llm("Is it true that the absence of a proof equates the proof of an absence?")

CPU times: user 11.2 ms, sys: 0 ns, total: 11.2 ms
Wall time: 714 ms


'\n\nNo, the two are not the same. Proof of absence would require evidence that something does not exist, while the absence of proof only means that there is no evidence to support something.'